In [1]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install numpy scipy sklearn



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from Sinkhorn_CMI import SinkhornImputation_CMI
from RR_imputer import RRimputer
import matplotlib.pyplot as plt

import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [3]:
s = pd.read_csv(r'C:\Users\zhossai3\Desktop\My code\OT+ Fairness\Data\Student_performance_data _.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\zhossai3\\Desktop\\My code\\OT+ Fairness\\Data\\Student_performance_data _.csv'

In [ ]:
print(s)

      Age  Gender  Ethnicity  ParentalEducation  StudyTimeWeekly  Absences  \
0      17       1          0                  2        19.833723         7   
1      18       0          0                  1        15.408756         0   
2      15       0          2                  3         4.210570        26   
3      17       1          0                  3        10.028829        14   
4      17       1          0                  2         4.672495        17   
...   ...     ...        ...                ...              ...       ...   
2387   18       1          0                  3        10.680555         2   
2388   17       0          0                  1         7.583217         4   
2389   16       1          0                  2         6.805500        20   
2390   16       1          1                  0        12.416653        17   
2391   16       1          0                  2        17.819907        13   

      Tutoring  ParentalSupport  Extracurricular  Sports  Music

In [ ]:
Xy = {}
with open(r'C:\Users\zhossai3\Desktop\My code\OT+ Fairness\Data\Student_performance_data _.csv', 'rb') as f:
        df = pd.read_csv(f, delimiter=',', header = 0)
        Xy['data'] = df.values[:, :-1]
        Xy['target'] =  df.values[:, -1]

In [ ]:
ground_truth_numpy = (Xy['data'])
print(type(ground_truth_numpy))

<class 'numpy.ndarray'>


In [ ]:
print(Xy['data'])

[[17.          1.          0.         ...  1.          0.
   2.92919559]
 [18.          0.          0.         ...  0.          0.
   3.04291483]
 [15.          0.          2.         ...  0.          0.
   0.11260225]
 ...
 [16.          1.          0.         ...  0.          1.
   1.14233288]
 [16.          1.          1.         ...  1.          0.
   1.80329676]
 [16.          1.          0.         ...  0.          1.
   2.14001388]]


In [ ]:
print(np.max(ground_truth_numpy))
print(np.min(ground_truth_numpy))
print(np.mean(ground_truth_numpy))
print(np.median(ground_truth_numpy))
print(np.std(ground_truth_numpy))

29.0
0.0
3.7913301488105686
1.0
6.288757915673482


In [ ]:
print(ground_truth_numpy)

[[17.          1.          0.         ...  1.          0.
   2.92919559]
 [18.          0.          0.         ...  0.          0.
   3.04291483]
 [15.          0.          2.         ...  0.          0.
   0.11260225]
 ...
 [16.          1.          0.         ...  0.          1.
   1.14233288]
 [16.          1.          1.         ...  1.          0.
   1.80329676]
 [16.          1.          0.         ...  0.          1.
   2.14001388]]


In [ ]:
print(ground_truth_numpy.shape)
ground_truth = torch.from_numpy(ground_truth_numpy)
print(ground_truth.isnan().sum())

(2392, 13)
tensor(0)


In [ ]:
data = ground_truth_numpy # Convert tensor to numpy array
sensitive_attributes = [1, 2]

# Define different bin sizes for each sensitive attribute
bin_sizes = {
    1: 2,  # Binary attribute
    2: 4  # More categories
}


# Digitize multiple sensitive attributes with different bin sizes
X_list = [
    np.digitize(data[:, col], bins=np.linspace(data[:, col].min(), data[:, col].max(), bin_sizes[col] + 1))
    for col in sensitive_attributes
]
X = np.column_stack(X_list)
#X = np.digitize(data[:,1],bins=np.linspace(data[:,1].min(), data[:,1].max(),1)) 
Y =np.digitize(Xy['target'],bins=np.linspace(Xy['target'].min(), Xy['target'].max(),4)) 
Z =  np.digitize(data[:, 7],bins=np.linspace(data[:, 7].min(), data[:, 7].max(), 4))


In [ ]:
from sklearn.metrics import mutual_info_score
import numpy as np

def conditional_mutual_info(X, Y, Z, min_samples=5):
    """
    Compute Conditional Mutual Information I(X; Y | Z)
    using mutual_info_score from sklearn.
    
    Parameters:
    - X, Y, Z: 1D NumPy arrays of the same length (categorical data)
    - min_samples: Minimum number of samples required per Z = z
    
    Returns:
    - CMI value
    """
    unique_Z = np.unique(Z)
    cmi = 0.0

    for z in unique_Z:
        mask = Z == z
        
        # Skip if not enough samples
        if np.sum(mask) < min_samples:
            continue
        
        # Skip if X or Y has no variance
        if len(np.unique(X[mask])) <= 1 or len(np.unique(Y[mask])) <= 1:
            continue

        # Compute MI for Z = z, weighted by P(Z = z)
        cmi += mutual_info_score(X[mask], Y[mask]) * np.mean(mask)
    
    return cmi


# Compute CMI
cmi_value = conditional_mutual_info(X, Y, Z)
print("Conditional Mutual Information:", cmi_value)


Conditional Mutual Information: 0.005112465952673926


In [ ]:
# Re-import libraries after reset
import pandas as pd
import numpy as np
from sklearn.metrics import mutual_info_score

# Load the uploaded student performance data



# Modified CMI function to handle multiple sensitive attributes
def conditional_mutual_info_multi_sensitive(X_list, Y, Z, min_samples=5):
    """
    Compute CMI for multiple sensitive attributes.
    
    Parameters:
    - X_list: List of sensitive attribute arrays (e.g., [Gender, Ethnicity])
    - Y: Target variable (GradeClass)
    - Z: Conditioning variable (Parental Support)
    - min_samples: Minimum number of samples required for each subgroup
    
    Returns:
    - CMI value
    """
    unique_Z = np.unique(Z)
    cmi = 0.0

    for z in unique_Z:
        mask = Z == z

        # Skip if not enough samples
        if np.sum(mask) < min_samples:
            continue

        # Combine sensitive attributes
        X_combined = np.column_stack([X[mask] for X in X_list])

        # Skip if X or Y has no variance
        if np.all([len(np.unique(X_combined[:, i])) <= 1 for i in range(X_combined.shape[1])]) or len(np.unique(Y[mask])) <= 1:
            continue

        # Compute MI for combined sensitive attributes
        mi = mutual_info_score(X_combined[:, 0], Y[mask])  # Using the first sensitive attribute
        for i in range(1, X_combined.shape[1]):
            mi += mutual_info_score(X_combined[:, i], Y[mask])

        weight = np.mean(mask)
        cmi += mi * weight

    return cmi



# Target and conditioning variable


# Compute CMI for multiple sensitive attributes
cmi_multi_sensitive = conditional_mutual_info_multi_sensitive(X_list, Y, Z)

cmi_multi_sensitive


np.float64(0.006568645808023421)

In [ ]:
#30% MCAR
np.random.seed(42)

p_macr30 = 0.301


mask_mcar30 = np.random.rand(*ground_truth.shape) < p_macr30  # True for missing values, false for others
data_mcar30 = np.copy(ground_truth)

# Apply the mask to set the selected values to NaN
data_mcar30[mask_mcar30] = np.nan

# Convert to a torch tensor
data_mcar30 = torch.from_numpy(data_mcar30)

# Calculate the percentage of missing values
missing_values_mcar30 = torch.isnan(data_mcar30)  # Create a boolean tensor where NaNs are True
missing_percentage_mcar30 = torch.sum(missing_values_mcar30).item() / data_mcar30.numel() * 100  # Calculate the percentage of NaNs
print(f"Percentage of missing values: {missing_percentage_mcar30:.2f}%")

Percentage of missing values: 29.96%


In [ ]:
data_mcar30_numpy = data_mcar30.detach().cpu().numpy()


In [ ]:
data_mcar30_numpy[0:,1]

array([ 1., nan,  0., ..., nan,  1., nan])

In [ ]:
#sinkhorn
n_mcar30, d_mcar30 = data_mcar30.shape
batchsize = 128 # If the batch size is larger than half the dataset's size,
                # it will be redefined in the imputation methods.
lr = 1e-2
epsilon_mcar30 = pick_epsilon(data_mcar30)
#epsilon_mcar30 = 0.3
print(epsilon_mcar30)


2.7581304589208884


In [ ]:
sk_imputer_mcar30 = SinkhornImputation_CMI(eps=epsilon_mcar30, batchsize=batchsize, lr=lr, niter=2000)
sk_imp_mcar30, sk_maes30mcar, sk_rmses30mcar,sk_loss_mcar30 = sk_imputer_mcar30.fit_transform(data_mcar30, verbose=True, report_interval=50, X_true=ground_truth)
#using numpy version of data
sk_imp_mcar30_numpy = sk_imp_mcar30.detach().cpu().numpy()

sk_mae_mcar30 = MAE(sk_imp_mcar30_numpy,ground_truth_numpy ,mask_mcar30)
sk_rmse_mcar30 = RMSE(sk_imp_mcar30_numpy, ground_truth_numpy,mask_mcar30)
print(sk_mae_mcar30,sk_rmse_mcar30)

INFO:root:Iteration 0:	 Loss: 2.0017	 Validation MAE: 1.4362	RMSE: 2.8862
INFO:root:Iteration 50:	 Loss: 2.6822	 Validation MAE: 1.4343	RMSE: 2.8846
INFO:root:Iteration 100:	 Loss: 1.7624	 Validation MAE: 1.4307	RMSE: 2.8823
INFO:root:Iteration 150:	 Loss: 3.0153	 Validation MAE: 1.4283	RMSE: 2.8825
INFO:root:Iteration 200:	 Loss: 3.8440	 Validation MAE: 1.4263	RMSE: 2.8821
INFO:root:Iteration 250:	 Loss: 2.9376	 Validation MAE: 1.4248	RMSE: 2.8822
INFO:root:Iteration 300:	 Loss: 3.1722	 Validation MAE: 1.4226	RMSE: 2.8813
INFO:root:Iteration 350:	 Loss: 2.0711	 Validation MAE: 1.4212	RMSE: 2.8807
INFO:root:Iteration 400:	 Loss: 2.8049	 Validation MAE: 1.4189	RMSE: 2.8799
INFO:root:Iteration 450:	 Loss: 4.9795	 Validation MAE: 1.4176	RMSE: 2.8798
INFO:root:Iteration 500:	 Loss: 3.4460	 Validation MAE: 1.4165	RMSE: 2.8800
INFO:root:Iteration 550:	 Loss: 2.5326	 Validation MAE: 1.4157	RMSE: 2.8796
INFO:root:Iteration 600:	 Loss: 5.1077	 Validation MAE: 1.4168	RMSE: 2.8801
INFO:root:Itera

1.410387068381239 2.8678257103720632


In [ ]:
sk_imp_mcar30

tensor([[17.0000,  1.0000,  0.0000,  ...,  0.0321,  0.0000,  2.9292],
        [16.4177,  0.5797,  0.8860,  ...,  0.0000,  0.0000,  3.0429],
        [16.5306,  0.0000,  2.0000,  ...,  0.0000,  0.1719,  0.1126],
        ...,
        [16.0000,  0.6995,  0.0000,  ...,  0.0000,  1.0000,  1.1423],
        [16.0000,  1.0000,  0.7378,  ...,  1.0000,  0.0000,  1.7813],
        [16.0000,  0.6411,  0.0000,  ...,  0.3265,  1.0000,  2.2147]],
       grad_fn=<IndexPutBackward0>)

In [ ]:
import numpy as np
data = sk_imp_mcar30.detach().numpy()  # Convert tensor to numpy array
sensitive_attributes = [1, 2]

# Define different bin sizes for each sensitive attribute
bin_sizes = {
    1: 2,  # Binary attribute
    2: 4  # More categories
}


# Digitize multiple sensitive attributes with different bin sizes
X_list = [
    np.digitize(data[:, col], bins=np.linspace(data[:, col].min(), data[:, col].max(), bin_sizes[col] + 1))
    for col in sensitive_attributes
]
X = np.column_stack(X_list)
#X = np.digitize(data[:,1],bins=np.linspace(data[:,1].min(), data[:,1].max(),2)) 
Y =np.digitize(Xy['target'],bins=np.linspace(Xy['target'].min(), Xy['target'].max(),4)) 
Z =  np.digitize(data[:, 7],bins=np.linspace(data[:, 7].min(), data[:, 7].max(), 4))


In [ ]:
from sklearn.metrics import mutual_info_score

def conditional_mutual_info(X, Y, Z):
    unique_Z = np.unique(Z)
    cmi = 0.0
    for z in unique_Z:
        mask = Z == z
        cmi += mutual_info_score(X[mask], Y[mask]) * np.mean(mask)
    return cmi

cmi_value = conditional_mutual_info(X, Y, Z)
print("Conditional Mutual Information:", cmi_value)

Conditional Mutual Information: 0.0024563764211416213


In [ ]:
import numpy as np
from sklearn.metrics import mutual_info_score



# Debugging: Check unique values
print("Unique X:", np.unique(X))
print("Unique Y:", np.unique(Y))
print("Unique Z:", np.unique(Z))

# Debugging: Check unconditional MI
mi = mutual_info_score(X, Y)
print("Mutual Information (unconditional):", mi)

# Debugging: Check subsets for each Z
for z in np.unique(Z):
    mask = Z == z
    print(f"Z = {z}:")
    print("X subset:", X[mask])
    print("Y subset:", Y[mask])
    print("Mutual Info:", mutual_info_score(X[mask], Y[mask]))

# Compute CMI
def conditional_mutual_info(X, Y, Z):
    unique_Z = np.unique(Z)
    cmi = 0.0
    for z in unique_Z:
        mask = Z == z
        if np.sum(mask) > 1 and len(np.unique(X[mask])) > 1 and len(np.unique(Y[mask])) > 1:
            mi = mutual_info_score(X[mask], Y[mask])
            cmi += mi * np.mean(mask)
    return cmi

cmi_value = conditional_mutual_info(X, Y, Z)

print("Conditional Mutual Information:", cmi_value)

Unique X: [1 2]
Unique Y: [1 2 3 4]
Unique Z: [1 2 3 4]
Mutual Information (unconditional): 0.0005774896490555737
Z = 1:
X subset: [1 1 1 2 1 1 1 1 1 2 1 2 1 1 1 1 2 2 1 1 1 1 1 2 2 1 1 2 2 2 2 2 1 2 1 2 1
 1 2 2 2 2 1 2 1 2 1 2 2 2 1 1 2 1 1 1 1 1 2 1 2 1 2 1 1 1 1 1 2 2 1 1 2 1
 1 2 2 2 1 1 2 2 1 1 1 2 1 2 2 1 1 1 1 1 1 1 2 2 1 1 2 1 1 2 1 1 1 1 2 1 2
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 2 1 1 1 1 2 2 2 1 1 2 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2 1 1 2 1 2 2 1 1 2
 1 2 1 1 2 2 1 2 2 1 2 2 1 2 1 1 1 2 2 1 1 2 2 1 2 2 1 1 2 2 1 2 2 2 1 2 1
 1 1 1 2 1 1 2 1 1 1 1 2 2 1 2 1 1 2 1 1 1 1 1 1 2 2 2 1 2 1 2 1 2 1 2 2 1
 2 1 1 2 2 1 1 2 1 1 1 1 2 2 1 2 1 2 2 1 1 1 2 2 2 1 1 1 1 1 1 2 1 1 2 1 1
 1 1 1 1 1 1 1 2 2 2 2 1 1 2 1 1 2 2 1 1 1 2 2 2 2 2 1 2 1 1 2 1 2 1 1 1 1
 1 2 1 1 1 1 2 2 1 1 1 1 1 2 1 1 2 2 2 1 2 2 2 2 2 2 2 1 1 2 1 2 1 1 1 1 2
 1 1 1 1 1 1 2 1 1 2 2 2 1 2 1 1 1 1 1 2 1 2 1 1 2 2 1 1 2 1 1 2 2 2 2 2 2
 2 1 2 2 1 2 1 1 1 1 1 1 1 1 1 2 1 1 2 1 2 1

In [ ]:
# Re-import libraries after reset
import pandas as pd
import numpy as np
from sklearn.metrics import mutual_info_score

# Load the uploaded student performance data



# Modified CMI function to handle multiple sensitive attributes
def conditional_mutual_info_multi_sensitive(X_list, Y, Z, min_samples=5):
    """
    Compute CMI for multiple sensitive attributes.
    
    Parameters:
    - X_list: List of sensitive attribute arrays (e.g., [Gender, Ethnicity])
    - Y: Target variable (GradeClass)
    - Z: Conditioning variable (Parental Support)
    - min_samples: Minimum number of samples required for each subgroup
    
    Returns:
    - CMI value
    """
    unique_Z = np.unique(Z)
    cmi = 0.0

    for z in unique_Z:
        mask = Z == z

        # Skip if not enough samples
        if np.sum(mask) < min_samples:
            continue

        # Combine sensitive attributes
        X_combined = np.column_stack([X[mask] for X in X_list])

        # Skip if X or Y has no variance
        if np.all([len(np.unique(X_combined[:, i])) <= 1 for i in range(X_combined.shape[1])]) or len(np.unique(Y[mask])) <= 1:
            continue

        # Compute MI for combined sensitive attributes
        mi = mutual_info_score(X_combined[:, 0], Y[mask])  # Using the first sensitive attribute
        for i in range(1, X_combined.shape[1]):
            mi += mutual_info_score(X_combined[:, i], Y[mask])

        weight = np.mean(mask)
        cmi += mi * weight

    return cmi



# Target and conditioning variable


# Compute CMI for multiple sensitive attributes
cmi_multi_sensitive = conditional_mutual_info_multi_sensitive(X_list, Y, Z)

cmi_multi_sensitive


np.float64(0.016868525956684213)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy

# Load the dataset



# Function to compute joint entropy
def joint_entropy(*args, bins=10):
    """
    Compute joint entropy of multiple variables.
    
    Parameters:
    - args: Arrays of variables (e.g., X, Y, Z)
    - bins: Number of bins for histogram (default: 10)
    
    Returns:
    - Joint entropy value
    """
    # Stack variables and compute histogram
    hist, _ = np.histogramdd(np.column_stack(args), bins=bins)
    prob = hist / np.sum(hist)  # Normalize to get probabilities
    prob = prob[prob > 0]  # Remove zero probabilities
    return entropy(prob, base=2)

def conditional_mutual_info_multi_sensitive(X, Y, Z, min_samples=5, bins=10):
    """
    Compute Conditional Mutual Information (CMI) for multiple sensitive attributes.
    
    Parameters:
    - X_list: List of sensitive attribute arrays (e.g., [Gender, Ethnicity])
    - Y: Target variable (e.g., GradeClass)
    - Z: Conditioning variable (e.g., Parental Support)
    - min_samples: Minimum number of samples required for each subgroup
    - bins: Number of bins for joint entropy calculation
    
    Returns:
    - CMI value
    """
    # Input validation
    
    unique_Z = np.unique(Z)
    cmi = 0.0

    for z in unique_Z:
        mask = (Z == z)

        # Skip if not enough samples
        if np.sum(mask) < min_samples:
            continue

        # Apply the mask to all sensitive attributes at once
        X_masked = X[mask]
        Y_masked = Y[mask]

        # Skip if X or Y has no variance
        if np.all([len(np.unique(X_masked[:, i])) <= 1 for i in range(X_masked.shape[1])]) or len(np.unique(Y_masked)) <= 1:
            continue

        # Compute MI using corrected joint entropy formula
        H_XZ = joint_entropy(X_masked, Z[mask])  # Fixed term
        H_YZ = joint_entropy(Y_masked, Z[mask])
        H_XYZ = joint_entropy(X_masked, Y_masked, Z[mask])
        H_Z = joint_entropy(Z[mask])  # Corrected term
        mi = (H_XZ + H_YZ - H_XYZ - H_Z)  # Fixed MI formula
        
        weight = np.mean(mask)
        cmi += mi * weight
    
    return max(0, cmi) 

# Define sensitive attributes (e.g., Gender and Ethnicity)


cmi_multi_sensitive = conditional_mutual_info_multi_sensitive(X, Y, Z)
print("Conditional Mutual Information (CMI) for multiple sensitive attributes:", cmi_multi_sensitive)


Conditional Mutual Information (CMI) for multiple sensitive attributes: 0.05089958732674174


In [ ]:
def conditional_mutual_information(data, X, Y, Z, delta=1):

  cmi = 0

  P_Z = data.groupby(Z).size()
  P_Z = P_Z / P_Z.sum()

  P_XZ = data.groupby(X + Z).size()
  P_XZ = P_XZ / P_XZ.sum()

  P_YZ = data.groupby(Y + Z).size()
  P_YZ = P_YZ / P_YZ.sum()

  P_XYZ = data.groupby(X + Y + Z).size()
  P_XYZ = P_XYZ / P_XYZ.sum()

  for ind in P_XYZ.index:
    x_ind = ind[:len(X)]
    y_ind = ind[len(X):len(X + Y)]
    z_ind = ind[len(X + Y):]

    xz_ind = x_ind + z_ind
    yz_ind = y_ind + z_ind
    xyz_ind = ind

    z_ind = pd.MultiIndex.from_tuples([z_ind], names=Z) if len(Z) != 1 else pd.Index(z_ind, name=Z[0])
    xz_ind = pd.MultiIndex.from_tuples([xz_ind], names=X + Z)
    yz_ind = pd.MultiIndex.from_tuples([yz_ind], names=Y + Z)
    xyz_ind = pd.MultiIndex.from_tuples([xyz_ind], names=X + Y + Z)

    cmi += delta * P_XYZ[xyz_ind].item() * np.log2(
      P_Z[z_ind].item() * P_XYZ[xyz_ind].item() / (P_XZ[xz_ind].item() * P_YZ[yz_ind].item()))

  return cmi

In [ ]:
mi_value =conditional_mutual_information(df, X, Y, Z, delta=1)
print("Conditional Mutual Information:", cmi_value)